# Updating UFC Stats

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     
import shutil       
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
import requests
import json
from random import randint
import  random
import os
from cmath import nan
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
import pickle
from sklearn.metrics import fbeta_score
import winsound
from bs4 import BeautifulSoup
import time


In [ ]:
def get_event_date_from_ufcstats(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    date = soup.find('li', class_='b-list__box-list-item').text
    date = date.replace('\n', '')
    ed = date.find(':')
    date = date[ed+2:].strip()
    return date

In [ ]:
def get_details_from_ufcstats(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    details = soup.find('div', class_='b-fight-details__content').text
    details = details.replace('\n', '')
    deet = details.find('Details:')
    details = details[deet+8:].strip()
    return details

In [ ]:
def get_event_title_from_ufcstats(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    title = soup.find('h2', class_='b-content__title').text
    title = title.replace('\n', '').strip()
    return title

In [ ]:
def get_event_url_from_ufcstats(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    title = soup.find('h2', class_='b-content__title')
    title = title.find('a')
    title = title['href']
    return title

In [ ]:
def get_fight_urls(urls):

    # get links to all fights on each event page

    links = pd.DataFrame()
    for u in urls:
        try:
            reqs = requests.get(u)
            soup = BeautifulSoup(reqs.text, 'html.parser')
            title = soup.find('h2', class_='b-content__title').text
            title = title.replace('\n', '')
            date = soup.find('li', class_='b-list__box-list-item').text
            d = date.find('Date:')
            date = date[d+6:].replace('\n', '')
            for link in soup.find_all('a'):
                # append url and event title to df
                if 'fight-details' in link.get('href'):
                    links = links.append({'Fight_url': link.get('href'), 'Event_title': title, 'Date' : date}, ignore_index=True)

        except:
            return 'No Fighties'
            
    return links

In [ ]:
def get_winner_from_ufcstats(url):

    ### This function scrapes the UFCStats website for the winner of a given fight###

    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        both = soup.find_all('div', class_='b-fight-details__person')
        first = both[0].text.replace('   ', '').replace('\n', '').strip()
        second = both[1].text.replace('   ', '').replace('\n', '').strip()
        if(first.startswith('W')):
            winner = first[3:]
        else:
            winner = second[3:]
        return winner
        
    except:
        return nan

In [ ]:
def get_event_date_from_ufcstats(url):
    ### This function scrapes the UFCStats website for the event date of a given fight ###
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        date = soup.find('li', class_='b-list__box-list-item').text
        date = date.replace('\n', '')
        ed = date.find(':')
        date = date[ed+2:].strip()
        return date
    except:
        return nan

In [ ]:
def get_event_details(event_id):
    # This function scrapes event details from ufc stats

    #try:
        df = pd.read_html('http://www.ufcstats.com/event-details/' + event_id)
        df = df[0]
        # split by space and get third element
        df['Fighter1'] = df['Fighter'].str.split('  ').str[0]
        df['Fighter2'] = df['Fighter'].str.split('  ').str[1]
        # fix Kd
        df['F1_Kd'] = df['Kd'].str.split('  ').str[0]
        df['F2_Kd'] = df['Kd'].str.split('  ').str[1]
        # fix Str
        df['F1_Str'] = df['Str'].str.split('  ').str[0]
        df['F2_Str'] = df['Str'].str.split('  ').str[1]
        # fix Td
        df['F1_Td'] = df['Td'].str.split('  ').str[0]
        df['F2_Td'] = df['Td'].str.split('  ').str[1]
        # fix Sub
        df['F1_Sub'] = df['Sub'].str.split('  ').str[0]
        df['F2_Sub'] = df['Sub'].str.split('  ').str[1]

        badcols = ['Fighter', 'Kd', 'Str', 'Td', 'Sub']

        # save index+1 to new column called "fight_num"
        df['fight_num'] = df.index + 1
        df['event_id'] = event_id

        # get fight links
        page = requests.get('http://www.ufcstats.com/event-details/' + event_id)
        soup = BeautifulSoup(page.content, 'html.parser')
        fight_linkies = soup.find_all('a')
        fight_links = [n['href'] for n in fight_linkies]
        fight_links = [n for n in fight_links if 'fight-details' in n]
        # delete duplicate links
        fight_links2 = pd.DataFrame(fight_links)
        fight_links2 = fight_links2.drop_duplicates()
        fight_links = fight_links2[0].tolist()

        # append fight links to df
        df['fight_link'] = fight_links

        df = df.drop(badcols, axis=1)
        df.to_csv('data/ufc_stats/events2/'+ event_id +'.csv')
        return df

In [ ]:
def get_fight_totals(fight_details_url):
    # function returns the fight totals for a given fightid
    da_url = fight_details_url
    fightid = da_url[da_url.find('fight-details/')+14:]
    df = pd.read_html(da_url)
    # Part 1: Total Fight Stats
    totals = df[0]
    # replace '---' with 0
    for col in totals.columns:
        totals[col] = totals[col].astype(str).replace({'---': 0})
        totals[col] = totals[col].astype(str).replace({'--': 0})


    totals['Fighter_A'] = totals['Fighter'].str.split(' ').str[0] + ' ' + totals['Fighter'].str.split(' ').str[1]
    totals['Fighter_B'] = totals['Fighter'].str.split(' ').str[2] + ' ' + totals['Fighter'].str.split(' ').str[3]
    totals['A_Kd'] = totals['KD'].str.split('  ').str[0]
    totals['B_Kd'] = totals['KD'].str.split('  ').str[1]
    totals['A_Sig_strike'] = totals['Sig. str.'].str.split('  ').str[0]
    totals['B_Sig_strike'] = totals['Sig. str.'].str.split('  ').str[1]
    # split sig strikes into landed and attempted
    totals['A_Sig_strike_land'] = totals['A_Sig_strike'].str.split(' of ').str[0]
    totals['A_Sig_strike_att'] = totals['A_Sig_strike'].str.split(' of ').str[1]
    totals['B_Sig_strike_land'] = totals['B_Sig_strike'].str.split(' of ').str[0]
    totals['B_Sig_strike_att'] = totals['B_Sig_strike'].str.split(' of ').str[1]
    # change to numeric
    totals['A_Sig_strike_land'].replace({'---', 0}, inplace=True)
    totals['A_Sig_strike_att'].replace({'---', 0}, inplace=True)
    totals['B_Sig_strike_land'].replace({'---', 0}, inplace=True)
    totals['B_Sig_strike_att'].replace({'---', 0}, inplace=True)

    totals['A_Sig_strike_land'] = pd.to_numeric(totals['A_Sig_strike_land'])
    totals['A_Sig_strike_att'] = pd.to_numeric(totals['A_Sig_strike_att'])
    totals['B_Sig_strike_land'] = pd.to_numeric(totals['B_Sig_strike_land'])
    totals['B_Sig_strike_att'] = pd.to_numeric(totals['B_Sig_strike_att'])

    totals['A_Sig_strike_percent'] = totals['Sig. str. %'].str.split('  ').str[0]
    totals['B_Sig_strike_percent'] = totals['Sig. str. %'].str.split('  ').str[1]
    # get rid of % sign
    totals['A_Sig_strike_percent'] = totals['A_Sig_strike_percent'].str.replace('%', '')
    totals['B_Sig_strike_percent'] = totals['B_Sig_strike_percent'].str.replace('%', '')

    totals['A_Sig_strike_percent'] = totals['A_Sig_strike_percent'].astype(str).replace({'---': 0})
    totals['B_Sig_strike_percent'] = totals['B_Sig_strike_percent'].astype(str).replace({'---': 0})
    # change to numeric
    totals['A_Sig_strike_percent'] = pd.to_numeric(totals['A_Sig_strike_percent'])/100
    totals['B_Sig_strike_percent'] = pd.to_numeric(totals['B_Sig_strike_percent'])/100
    # total strikes
    totals['A_Total_Strikes'] = totals['Total str.'].str.split('  ').str[0]
    totals['B_Total_Strikes'] = totals['Total str.'].str.split('  ').str[1]
    # split total strikes into landed and attempted
    totals['A_Total_Strikes_land'] = totals['A_Total_Strikes'].str.split(' of ').str[0]
    totals['A_Total_Strikes_att'] = totals['A_Total_Strikes'].str.split(' of ').str[1]
    totals['B_Total_Strikes_land'] = totals['B_Total_Strikes'].str.split(' of ').str[0]
    totals['B_Total_Strikes_att'] = totals['B_Total_Strikes'].str.split(' of ').str[1]

    totals['A_Total_Strikes_land'].replace({'---',0}, inplace=True)
    totals['A_Total_Strikes_att'].replace({'---',0}, inplace=True)
    totals['B_Total_Strikes_land'].replace({'---',0}, inplace=True)
    totals['B_Total_Strikes_att'].replace({'---',0}, inplace=True)
    # change to numeric
    totals['A_Total_Strikes_land'] = pd.to_numeric(totals['A_Total_Strikes_land'])
    totals['A_Total_Strikes_att'] = pd.to_numeric(totals['A_Total_Strikes_att'])
    totals['B_Total_Strikes_land'] = pd.to_numeric(totals['B_Total_Strikes_land'])
    totals['B_Total_Strikes_att'] = pd.to_numeric(totals['B_Total_Strikes_att'])

    totals['A_Total_Strikes_land'].astype(str).replace({'---',0}, inplace=True)
    totals['A_Total_Strikes_att'].astype(str).replace({'---',0}, inplace=True)
    totals['B_Total_Strikes_land'].astype(str).replace({'---',0}, inplace=True)
    totals['B_Total_Strikes_att'].astype(str).replace({'---',0}, inplace=True) 

    # calculate total strike percentage
    totals['A_Total_Strikes_percent'] = totals['A_Total_Strikes_land'] / totals['A_Total_Strikes_att']
    totals['B_Total_Strikes_percent'] = totals['B_Total_Strikes_land'] / totals['B_Total_Strikes_att']
    # takedown stats
    totals['A_Takedowns'] = totals['Td'].str.split('  ').str[0]
    totals['B_Takedowns'] = totals['Td'].str.split('  ').str[1]
    # split takedowns into landed and attempted
    totals['A_Takedowns_land'] = totals['A_Takedowns'].str.split(' of ').str[0]
    totals['A_Takedowns_att'] = totals['A_Takedowns'].str.split(' of ').str[1]
    totals['B_Takedowns_land'] = totals['B_Takedowns'].str.split(' of ').str[0]
    totals['B_Takedowns_att'] = totals['B_Takedowns'].str.split(' of ').str[1]
    # change to numeric
    totals['A_Takedowns_land'] = pd.to_numeric(totals['A_Takedowns_land'])
    totals['A_Takedowns_att'] = pd.to_numeric(totals['A_Takedowns_att'])
    totals['B_Takedowns_land'] = pd.to_numeric(totals['B_Takedowns_land'])
    totals['B_Takedowns_att'] = pd.to_numeric(totals['B_Takedowns_att'])
    # fix %
    totals['Td %'].replace({'---': 0}, inplace=True)
    # get rid of ---
    totals['A_Takedowns_land'].replace({'---': 0}, inplace=True)
    totals['A_Takedowns_att'].replace({'---': 0}, inplace=True)
    totals['B_Takedowns_land'].replace({'---': 0}, inplace=True)
    totals['B_Takedowns_att'].replace({'---': 0}, inplace=True)
    totals['A_Takedown_percent'] = totals['A_Takedowns_land'] / totals['A_Takedowns_att']
    totals['B_Takedown_percent'] = totals['B_Takedowns_land'] / totals['B_Takedowns_att']
    # submission attempts
    totals['A_Sub_Attempts'] = totals['Sub. att'].str.split('  ').str[0]
    totals['B_Sub_Attempts'] = totals['Sub. att'].str.split('  ').str[1]
    # split submission attempts into landed and attempted
    totals['A_Sub_Attempts_land'] = totals['A_Sub_Attempts'].str.split(' of ').str[0]
    totals['A_Sub_Attempts_att'] = totals['A_Sub_Attempts'].str.split(' of ').str[1]
    totals['B_Sub_Attempts_land'] = totals['B_Sub_Attempts'].str.split(' of ').str[0]
    totals['B_Sub_Attempts_att'] = totals['B_Sub_Attempts'].str.split(' of ').str[1]
    # change to numeric
    totals['A_Sub_Attempts_land'] = pd.to_numeric(totals['A_Sub_Attempts_land'])
    totals['A_Sub_Attempts_att'] = pd.to_numeric(totals['A_Sub_Attempts_att'])
    totals['B_Sub_Attempts_land'] = pd.to_numeric(totals['B_Sub_Attempts_land'])
    totals['B_Sub_Attempts_att'] = pd.to_numeric(totals['B_Sub_Attempts_att'])
    # Rev
    totals['A_Rev'] = totals['Rev.'].str.split('  ').str[0]
    totals['B_Rev'] = totals['Rev.'].str.split('  ').str[1]
    # change to numeric
    totals['A_Rev'] = pd.to_numeric(totals['A_Rev'])
    totals['B_Rev'] = pd.to_numeric(totals['B_Rev'])
    # control time
    totals['A_Ctrl_time'] = totals['Ctrl'].str.split('  ').str[0]
    totals['B_Ctrl_time'] = totals['Ctrl'].str.split('  ').str[1]
    

    # split control time into minutes and seconds, and then convert to seconds
    totals['A_Ctrl_time_min'] = totals['A_Ctrl_time'].str.split(':').str[0]
    totals['A_Ctrl_time_sec'] = totals['A_Ctrl_time'].str.split(':').str[1]
    totals['B_Ctrl_time_min'] = totals['B_Ctrl_time'].str.split(':').str[0]
    totals['B_Ctrl_time_sec'] = totals['B_Ctrl_time'].str.split(':').str[1]

    totals['A_Ctrl_time_min'] = totals['A_Ctrl_time_min'].replace({'--': 0})
    totals['A_Ctrl_time_sec'] = totals['A_Ctrl_time_sec'].replace({'--': 0})
    totals['B_Ctrl_time_min'] = totals['B_Ctrl_time_min'].replace({'--': 0})
    totals['B_Ctrl_time_sec'] = totals['B_Ctrl_time_sec'].replace({'--': 0})

    # convert to seconds
    totals['A_Ctrl_time_tot'] = pd.to_numeric(totals['A_Ctrl_time_min']) * 60 + pd.to_numeric(totals['A_Ctrl_time_sec'])
    totals['B_Ctrl_time_tot'] = pd.to_numeric(totals['B_Ctrl_time_min']) * 60 + pd.to_numeric(totals['B_Ctrl_time_sec'])


    # add date with beatufil soup
    totals['details'] = get_details_from_ufcstats(da_url)
    totals['event_title'] = get_event_title_from_ufcstats(da_url)
    event_url = get_event_url_from_ufcstats(da_url)
    totals['event_url'] = event_url
    totals['date'] = get_event_date_from_ufcstats(event_url)

    try:
        totals['Winner']= get_winner_from_ufcstats(da_url)
    except:
        totals['Winner']= nan

    bad_cols = ['Fighter', 'KD', 'Sig. str. %', 'Sig. str.', 'Total str.', 'Td', 'Td %', 'Sub. att', 
                'Rev.', 'Ctrl', 'A_Sig_strike', 'B_Sig_strike', 'A_Takedowns', 'B_Takedowns',
                'A_Sub_Attempts', 'B_Sub_Attempts', 'A_Ctrl_time', 'B_Ctrl_time']
    totals.drop(columns=bad_cols, inplace=True)
    totals.to_csv('data/ufc_stats/fight_totals2/' + fightid + '_totals.csv')
    return totals
    


In [ ]:
def get_significant_strikes(url):
    df = pd.read_html(url)
    # Part 2: Significant Strikes
    ss = df[2]
     
    ss['Fighter_A'] = ss['Fighter'].str.split(' ').str[0] + ' ' + ss['Fighter'].str.split(' ').str[1]
    ss['Fighter_B'] = ss['Fighter'].str.split(' ').str[2] + ' ' + ss['Fighter'].str.split(' ').str[3]
    ss['A_Head_Strikes'] = ss['Head'].str.split('  ').str[0]
    ss['B_Head_Strikes'] = ss['Head'].str.split('  ').str[1]
    # split head strikes into landed and attempted
    ss['A_Head_Strikes_land'] = ss['A_Head_Strikes'].str.split(' of ').str[0]
    ss['A_Head_Strikes_att'] = ss['A_Head_Strikes'].str.split(' of ').str[1]
    ss['B_Head_Strikes_land'] = ss['B_Head_Strikes'].str.split(' of ').str[0]
    ss['B_Head_Strikes_att'] = ss['B_Head_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Head_Strikes_land'] = pd.to_numeric(ss['A_Head_Strikes_land'])
    ss['A_Head_Strikes_att'] = pd.to_numeric(ss['A_Head_Strikes_att'])
    ss['B_Head_Strikes_land'] = pd.to_numeric(ss['B_Head_Strikes_land'])
    ss['B_Head_Strikes_att'] = pd.to_numeric(ss['B_Head_Strikes_att'])
    # Head Strikes Percentage
    ss['A_Head_Strikes_percent'] = ss['A_Head_Strikes_land'] / ss['A_Head_Strikes_att']
    ss['B_Head_Strikes_percent'] = ss['B_Head_Strikes_land'] / ss['B_Head_Strikes_att']
    # Body Strikes
    ss['A_Body_Strikes'] = ss['Body'].str.split('  ').str[0]
    ss['B_Body_Strikes'] = ss['Body'].str.split('  ').str[1]
    # split body strikes into landed and attempted
    ss['A_Body_Strikes_land'] = ss['A_Body_Strikes'].str.split(' of ').str[0]
    ss['A_Body_Strikes_att'] = ss['A_Body_Strikes'].str.split(' of ').str[1]
    ss['B_Body_Strikes_land'] = ss['B_Body_Strikes'].str.split(' of ').str[0]
    ss['B_Body_Strikes_att'] = ss['B_Body_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Body_Strikes_land'] = pd.to_numeric(ss['A_Body_Strikes_land'])
    ss['A_Body_Strikes_att'] = pd.to_numeric(ss['A_Body_Strikes_att'])
    ss['B_Body_Strikes_land'] = pd.to_numeric(ss['B_Body_Strikes_land'])
    ss['B_Body_Strikes_att'] = pd.to_numeric(ss['B_Body_Strikes_att'])
    # Body Strikes Percentage
    ss['A_Body_Strikes_percent'] = ss['A_Body_Strikes_land'] / ss['A_Body_Strikes_att']
    ss['B_Body_Strikes_percent'] = ss['B_Body_Strikes_land'] / ss['B_Body_Strikes_att']
    # Leg Strikes
    ss['A_Leg_Strikes'] = ss['Leg'].str.split('  ').str[0]
    ss['B_Leg_Strikes'] = ss['Leg'].str.split('  ').str[1]
    # split leg strikes into landed and attempted
    ss['A_Leg_Strikes_land'] = ss['A_Leg_Strikes'].str.split(' of ').str[0]
    ss['A_Leg_Strikes_att'] = ss['A_Leg_Strikes'].str.split(' of ').str[1]
    ss['B_Leg_Strikes_land'] = ss['B_Leg_Strikes'].str.split(' of ').str[0]
    ss['B_Leg_Strikes_att'] = ss['B_Leg_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Leg_Strikes_land'] = pd.to_numeric(ss['A_Leg_Strikes_land'])
    ss['A_Leg_Strikes_att'] = pd.to_numeric(ss['A_Leg_Strikes_att'])
    ss['B_Leg_Strikes_land'] = pd.to_numeric(ss['B_Leg_Strikes_land'])
    ss['B_Leg_Strikes_att'] = pd.to_numeric(ss['B_Leg_Strikes_att'])
    # Leg Strikes Percentage
    ss['A_Leg_Strikes_percent'] = ss['A_Leg_Strikes_land'] / ss['A_Leg_Strikes_att']
    ss['B_Leg_Strikes_percent'] = ss['B_Leg_Strikes_land'] / ss['B_Leg_Strikes_att']
    # Distance Strikes
    ss['A_Distance_Strikes'] = ss['Distance'].str.split('  ').str[0]
    ss['B_Distance_Strikes'] = ss['Distance'].str.split('  ').str[1]
    # split distance strikes into landed and attempted
    ss['A_Distance_Strikes_land'] = ss['A_Distance_Strikes'].str.split(' of ').str[0]
    ss['A_Distance_Strikes_att'] = ss['A_Distance_Strikes'].str.split(' of ').str[1]
    ss['B_Distance_Strikes_land'] = ss['B_Distance_Strikes'].str.split(' of ').str[0]
    ss['B_Distance_Strikes_att'] = ss['B_Distance_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Distance_Strikes_land'] = pd.to_numeric(ss['A_Distance_Strikes_land'])
    ss['A_Distance_Strikes_att'] = pd.to_numeric(ss['A_Distance_Strikes_att'])
    ss['B_Distance_Strikes_land'] = pd.to_numeric(ss['B_Distance_Strikes_land'])
    ss['B_Distance_Strikes_att'] = pd.to_numeric(ss['B_Distance_Strikes_att'])
    # distance strikes percentage
    ss['A_Distance_Strikes_percent'] = ss['A_Distance_Strikes_land'] / ss['A_Distance_Strikes_att']
    ss['B_Distance_Strikes_percent'] = ss['B_Distance_Strikes_land'] / ss['B_Distance_Strikes_att']

    # Clinch Strikes
    ss['A_Clintch_Strikes'] = ss['Clinch'].str.split('  ').str[0]
    ss['B_Clintch_Strikes'] = ss['Clinch'].str.split('  ').str[1]
    # split clinch strikes into landed and attempted
    ss['A_Clinch_Strikes_land'] = ss['A_Clintch_Strikes'].str.split(' of ').str[0]
    ss['A_Clinch_Strikes_att'] = ss['A_Clintch_Strikes'].str.split(' of ').str[1]
    ss['B_Clinch_Strikes_land'] = ss['B_Clintch_Strikes'].str.split(' of ').str[0]
    ss['B_Clinch_Strikes_att'] = ss['B_Clintch_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Clinch_Strikes_land'] = pd.to_numeric(ss['A_Clinch_Strikes_land'])
    ss['A_Clinch_Strikes_att'] = pd.to_numeric(ss['A_Clinch_Strikes_att'])
    ss['B_Clinch_Strikes_land'] = pd.to_numeric(ss['B_Clinch_Strikes_land'])
    ss['B_Clinch_Strikes_att'] = pd.to_numeric(ss['B_Clinch_Strikes_att'])
    # clinch strikes percentage
    ss['A_Clinch_Strikes_percent'] = ss['A_Clinch_Strikes_land'] / ss['A_Clinch_Strikes_att']
    ss['B_Clinch_Strikes_percent'] = ss['B_Clinch_Strikes_land'] / ss['B_Clinch_Strikes_att']

    # Ground Strikes
    ss['A_Ground_Strikes'] = ss['Ground'].str.split('  ').str[0]
    ss['B_Ground_Strikes'] = ss['Ground'].str.split('  ').str[1]
    # split ground strikes into landed and attempted
    ss['A_Ground_Strikes_land'] = ss['A_Ground_Strikes'].str.split(' of ').str[0]
    ss['A_Ground_Strikes_att'] = ss['A_Ground_Strikes'].str.split(' of ').str[1]
    ss['B_Ground_Strikes_land'] = ss['B_Ground_Strikes'].str.split(' of ').str[0]
    ss['B_Ground_Strikes_att'] = ss['B_Ground_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Ground_Strikes_land'] = pd.to_numeric(ss['A_Ground_Strikes_land'])
    ss['A_Ground_Strikes_att'] = pd.to_numeric(ss['A_Ground_Strikes_att'])
    ss['B_Ground_Strikes_land'] = pd.to_numeric(ss['B_Ground_Strikes_land'])
    ss['B_Ground_Strikes_att'] = pd.to_numeric(ss['B_Ground_Strikes_att'])
    # ground strikes percentage
    ss['A_Ground_Strikes_percent'] = ss['A_Ground_Strikes_land'] / ss['A_Ground_Strikes_att']
    ss['B_Ground_Strikes_percent'] = ss['B_Ground_Strikes_land'] / ss['B_Ground_Strikes_att']
    # add stuff with beautiful soup
    ss['details'] = get_details_from_ufcstats(url)
    ss['event_title'] = get_event_title_from_ufcstats(url)
    event_url = get_event_url_from_ufcstats(url)
    ss['event_url'] = event_url

    fightid = url[url.find('fight-details/')+14:]

    bad_cols = ['Fighter', 'Sig. str', 'Sig. str. %', 'Head', 'Body', 'Leg', 'Distance', 'Clinch', 'Ground',
                'A_Head_Strikes', 'B_Head_Strikes', 'A_Body_Strikes', 'B_Body_Strikes', 'A_Leg_Strikes', 'B_Leg_Strikes',
                'A_Distance_Strikes', 'B_Distance_Strikes', 'A_Clintch_Strikes', 'B_Clintch_Strikes', 'A_Ground_Strikes', 
                'B_Ground_Strikes']
    ss.drop(columns=bad_cols, inplace=True)
    ss.to_csv('data/ufc_stats/sig_strikes2/' + fightid + '_sigstrikes.csv')

    return ss

## Update Event Data (check against files)

In [ ]:
# load all_event_data
all_event_data = pd.read_csv('data/final/events/All_Events_Fights_and_FightUrls.csv')

In [ ]:
inital_events_chosen = all_event_data['event_id'].unique()
print(len(inital_events_chosen))

In [ ]:
inital_events_chosen = inital_events_chosen.tolist()

In [ ]:
def get_most_recent_events():
    ufc_stats_events = "http://ufcstats.com/statistics/events/completed?page=all"

    # for each event, get the event link
    page = requests.get(ufc_stats_events)
    soup = BeautifulSoup(page.content, 'html.parser')
    event_linkies = soup.find_all('a', class_='b-link b-link_style_black')

    # list of event links and event titles
    event_links = [n['href'] for n in event_linkies]
    event_titles = [n.text for n in event_linkies]
    # replace '\n' with ''
    event_titles = [n.replace('\n', '').strip() for n in event_titles]

    # make df
    event_df = pd.DataFrame({'event_title': event_titles, 'event_url': event_links})
    # take most recent 5
    ed = event_df.head(5)

    # return 5 most recent
    return ed

In [ ]:
most_recent_events = get_most_recent_events()
most_recent_events

In [ ]:
# add event_id
most_recent_events['event_id'] = most_recent_events['event_url'].str.split('/').str[-1]

In [ ]:
completed_ufcstats_event_ids = most_recent_events['event_id'].tolist()

In [ ]:
def check_for_missing_events(completed_ufcstats_event_ids):
    
    # check the list of completed events against the events2 folder
    # make list of events that are no downloading

    missing = []
    for event in completed_ufcstats_event_ids:
        if event + '.csv' not in os.listdir('data/ufc_stats/events2'):
            missing.append(event)

    return missing


In [ ]:
missing_events = check_for_missing_events(completed_ufcstats_event_ids)
missing_events

In [ ]:
# just this one time
missing_events = missing_events[1:]

In [ ]:
# update by downloading events
for event in missing_events:
    get_event_details(event)

#### For each missing event, download all fights data

In [ ]:
def get_ufcstats_fight_urls(missing_event_df):

    # For each missing event, download the fight url links

    links = pd.DataFrame()
    for event in missing_event_df:
        data = pd.read_csv('data/ufc_stats/events2/'+ event +'.csv')
        links = links.append(data[['Fighter1', 'Fighter2','fight_link', 'event_id']], ignore_index=True)
    return links


In [ ]:
missing_fights = get_ufcstats_fight_urls(missing_events)
missing_fights

In [ ]:
if len(missing_fights) > 0:
    missing_fight_links = missing_fights['fight_link'].tolist()
    for fight_link in missing_fight_links:
        
        # get totals
        get_fight_totals(fight_link)
        get_significant_strikes(fight_link)

else:
    print('No missing fights')

## Update for NEXT Fight

In [ ]:
# function to return the next 3 UFC events using BeautifulSoup
def get_next_events(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    event1 = soup.find('div', class_='c-card-event--result__info')
    event1_txt = soup.find('div', class_='c-card-event--result__info').text
    event1_url = event1.find('a')['href']
    event1_url = 'https://www.ufc.com' + event1_url
    event1_title = event1_txt.split('\n')[1]
    event1_time = event1_txt.split('/')[1]

    data = pd.DataFrame({'event_title': [event1_title], 'event_url': [event1_url], 'event_date': [event1_time]})

    event2 = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info')
    event2_txt = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').text
    event2_url = event2.find('a')['href']
    event2_url = 'https://www.ufc.com' + event2_url
    event2_title = event2_txt.split('\n')[1]
    event2_time = event2_txt.split('/')[1]


    data = data.append({'event_title': event2_title, 'event_url': event2_url, 'event_date': event2_time}, ignore_index=True)
    
    event3 = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info')
    event3_txt = soup.find('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').find_next('div', class_='c-card-event--result__info').text
    event3_url = event3.find('a')['href']
    event3_url = 'https://www.ufc.com' + event3_url
    event3_title = event3_txt.split('\n')[1]
    event3_time = event3_txt.split('/')[1]

    data = data.append({'event_title': event3_title, 'event_url': event3_url, 'event_date': event3_time}, ignore_index=True)
    
    return data



# Function to get the fight card for a given event using BeautifulSoup
def get_event_fights(event_url):
    page = requests.get(event_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get main card, fight 1

    mcn = soup.find_all('li', class_='l-listing__item')
    # get num of mc
    num_mc = len(mcn)
    # for each mc, do the following
    data = pd.DataFrame()
    n = 0
    for i in mcn:
        mc = mcn[n]
        # fight 1
        fighter1= mc.find('div', class_ ='c-listing-fight__corner-name c-listing-fight__corner-name--red').text
        fighter1 = fighter1.replace('\n', ' ')
        fighter1 = fighter1.strip()
        fighter2 = mc.find('div', class_ ='c-listing-fight__corner-name c-listing-fight__corner-name--blue').text
        fighter2 = fighter2.replace('\n', ' ')
        fighter2 = fighter2.strip()
        weightclass = mc.find('div', class_='c-listing-fight__class-text').text
        fighter1_odds = mc.find('span', class_='c-listing-fight__odds').text
        fighter2_odds = mc.find('span', class_='c-listing-fight__odds').find_next('span', class_='c-listing-fight__odds').text
        fighter1_odds = fighter1_odds.replace('\n', '')
        fighter2_odds = fighter2_odds.replace('\n', '')
        # fighter odds to float
        if (fighter1_odds == '-') :
            fighter1_odds = nan
        if (fighter2_odds == '-') :
            fighter2_odds = nan

        data = data.append({'fighter1': fighter1, 'fighter2': fighter2, 'weightclass': weightclass, 
                            'fighter1_odds': fighter1_odds, 'fighter2_odds': fighter2_odds}, ignore_index=True)
        n = n + 1
    return data


# get next events if event fighter data is not na
def get_next_events2(url):
    data = get_next_events(url)
    for i in range(0, len(data)):
        event_url = data['event_url'][i]
        event_fights = get_event_fights(event_url)
        if (len(event_fights) == 0):
            data = data.drop(i)
    return data

# get next events from UFCStats.com using BS
def get_next_event_ufcstats():
    url = 'http://www.ufcstats.com/statistics/events/upcoming'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    event1 = soup.find('td', class_='b-statistics__table-col')
    event1_txt = soup.find('td', class_='b-statistics__table-col').text
    event_txt = event1_txt.replace('   ', '').replace('\n', '').strip()
    event_title = event_txt.split('  ')[0]
    event_date = event_txt.split('  ')[1]
    event1_url = event1.find('a')['href']
    data = pd.DataFrame({'event_title': [event_title], 'event_url': [event1_url], 'event_date': [event_date]})
    return data


# get fighter urls from UFCStats.com using BS
def get_fighter_urls(event_details_url):
    page = requests.get(event_details_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    events = soup.find_all('tr', class_='b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click')
    n = 0
    next_event_data = pd.DataFrame()

    for event in events:
        fighters = events[n].find_all('p', class_='b-fight-details__table-text')
        fighter1 = fighters[0].text
        fighter1 = fighter1.replace('  ', '').replace('\n', '').strip()
        fighter2 = fighters[1].text
        fighter2 = fighter2.replace('  ', '').replace('\n', '').strip()
        fighter1_url = fighters[0].find('a')['href']
        fighter2_url = fighters[1].find('a')['href']
        next_event_data = next_event_data.append({'fighter1' :fighter1, 'fighter2:' : fighter2, 'fighter1_url': fighter1_url, 'fighter2_url':fighter2_url, 'fight#' : n+1}, ignore_index = True)
        n += 1

    return next_event_data



In [ ]:
next_ufc_events = get_next_events('https://www.ufc.com/events')
next_ufc_events

Get event Fights

In [ ]:
# get event fights for first event
event_url = next_ufc_events['event_url'][0]
event_fights = get_event_fights(event_url)
event_fights

Get secret number

In [ ]:
# Find secret number in ufc events using BS & Selenium
def get_secret_number(event_url):
    # if no driver open, open one
    driver = None
    if (driver == None):
        driver = webdriver.Chrome('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\UFC_Prediction_V2\\chromedriver.exe')
    
    driver.get(event_url)
    # sleep for 15 seconds
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pretty = soup.prettify()

    # find first data-fmid to get first matchup
    fmid_start = pretty.find('data-fmid')
    fmid = pretty[fmid_start+11:fmid_start+16]

    # go to event url with fmid
    driver.get(event_url +'#' + fmid)
    time.sleep(8)

    # find all links within page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # find all iframe src
    iframe = soup.find_all('iframe')

    # find all links
    iframe_text = str(iframe)
    matchup = iframe_text.find('matchup')
    # matchup_url = iframe_text[matchup+8:matchup+12]
    # print('matchup_url: ' + matchup_url)
    # secret_number = matchup_url
    return matchup, fmid

In [ ]:
secret_number_1, fmid_1 = get_secret_number(event_url)
print('secret_number_1: ' + str(secret_number_1))
print('fmid_1: ' + str(fmid_1))


In [ ]:
url = 'https://www.ufc.com/matchup/' + str(secret_number_1) + '/' + str(fmid_1)

In [ ]:
def just_get_soup(matchup_url):
    response = requests.get(matchup_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

Get matchup data

In [ ]:
def grab_soup(match_url):
    # if no driver open, open one
    driver = None
    if (driver == None):
        driver = webdriver.Chrome('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\UFC_Prediction_V2\\chromedriver.exe')
    
    driver.get(match_url)
    # sleep for 5 seconds
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # find class="c-listing-fight__details-content details-content"

    #chosen_fight = soup.find('div', class_='c-listing-fight__details-content details-content')
    
    # get inner html
    inner_html = driver.execute_script("return document.body.innerHTML")
    chosen_fight = inner_html.find('c-listing-fight__details-content details-content')
    chosen_fight = inner_html[chosen_fight:chosen_fight+1000]


    # prettify inner html
    soup2 = BeautifulSoup(inner_html, 'html.parser')
    pretty = soup2.prettify()


    return chosen_fight

In [ ]:
# Function to scrape UFC fight data
def grab_matchup_data(matchup_url):
    response = requests.get(matchup_url)
    soup = BeautifulSoup(response.text, 'html.parser').text
    soup = soup.replace('   ', '').replace('\n', '')

    od = soup.find('Odds')
    rec = soup.find('Record')
    a_record = soup[od + 5 : rec - 2]
    last_fight = soup.find("Last Fight")
    b_record = soup[rec + 7 : last_fight - 5]

    hite = soup.find('Height')
    f = soup.find("' ")
    a_height = soup[f -1 : hite - 2]
    # find second occurance of f
    f2 = soup.find("' ", f + 1)
    b_height = soup[hite + 7 : f2+5]

    # Find reach
    reach = soup.find('Reach')
    # find second occurance of "LB"
    lb = soup.find('LB')
    lb2 = soup.find('LB', lb + 1)
    a_reach = soup[lb2 +5 : reach ]
    inn = soup.find("in ")
    # get the word after reach
    big_space = soup.find('  ', reach + 1)
    b_reach = soup[reach + 6 : big_space]

    # Find Leg Reach
    leg = soup.find('Leg Reach')
    big_space2 = soup.find('  ', big_space + 1)
    a_leg = soup[big_space2 + 2 : leg]
    big_space4 = soup.find('  ', big_space2 + 2)
    b_leg = soup[leg + 10 : leg + 17]

    a_record = a_record.strip()
    b_record = b_record.strip()

    a_height_ft = float(a_height[:1])
    a_height_in = float(a_height[3:].replace("'", "").replace('"', ''))
    a_height = (a_height_ft * 12) + a_height_in 


    b_height_ft = float(b_height[:1])
    b_height_in = float(b_height[3:].replace("'", "").replace('"', ''))
    b_height = (b_height_ft * 12) + b_height_in

    a_reach = float(a_reach.replace(' in', '').strip())
    b_reach = float(b_reach.replace(' in', '').strip())

    a_leg = float(a_leg.replace(' in', '').strip())
    b_leg = float(b_leg.replace(' in', '').strip())

    
    return a_record, b_record, a_height, b_height, a_reach, b_reach, a_leg, b_leg

In [ ]:
# test
matchup_url = 'https://www.ufc.com/event/ufc-287#10377'
a_record, b_record, a_height, b_height, a_reach, b_reach, a_leg, b_leg = grab_matchup_data(matchup_url)